In [1]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import wandb
import os

/Users/kyle/miniconda3/envs/eval-aug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
project_name = "kyledevinobrien1/In-Context Domain Transfer Improves Out-of-Domain Robustness"
table_paths = {
    # BOSS Sentiment
    # ID
    # BERT
    "BOSS_Sentiment_ID_BERT_Insert": "",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "",
    "BOSS_Sentiment_ID_BERT_ICR": "",

    # T5
    "BOSS_Sentiment_ID_T5_Insert": "",
    "BOSS_Sentiment_ID_T5_Substitute": "",
    "BOSS_Sentiment_ID_T5_Translate": "",
    "BOSS_Sentiment_ID_T5_Paraphrase": "",
    "BOSS_Sentiment_ID_T5_ICR": "",

    # Falcon
    "BOSS_Sentiment_ID_Falcon_Insert": "",
    "BOSS_Sentiment_ID_Falcon_Substitute": "",
    "BOSS_Sentiment_ID_Falcon_Translate": "",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": "",
    "BOSS_Sentiment_ID_Falcon_ICR": "",

    # SST5
    # BERT
    "BOSS_Sentiment_SST5_BERT_Insert": "",
    "BOSS_Sentiment_SST5_BERT_Substitute": "",
    "BOSS_Sentiment_SST5_BERT_Translate": "",
    "BOSS_Sentiment_SST5_BERT_Paraphrase": "",
    "BOSS_Sentiment_SST5_BERT_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT12000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_ICR": "",

    # T5
    "BOSS_Sentiment_SST5_T5_Insert": "",
    "BOSS_Sentiment_SST5_T5_Substitute": "",
    "BOSS_Sentiment_SST5_T5_Translate": "",
    "BOSS_Sentiment_SST5_T5_Paraphrase": "",
    "BOSS_Sentiment_SST5_T5_ICR": "",

    # Falcon
    "BOSS_Sentiment_SST5_Falcon_Insert": "",
    "BOSS_Sentiment_SST5_Falcon_Substitute": "",
    "BOSS_Sentiment_SST5_Falcon_Translate": "",
    "BOSS_Sentiment_SST5_Falcon_Paraphrase": "",
    "BOSS_Sentiment_SST5_Falcon_ICR": "",

    # SemEval
    # BERT
    "BOSS_Sentiment_SemEval_BERT_Insert": "",
    "BOSS_Sentiment_SemEval_BERT_Substitute": "",
    "BOSS_Sentiment_SemEval_BERT_Translate": "",
    "BOSS_Sentiment_SemEval_BERT_Paraphrase": "",
    "BOSS_Sentiment_SemEval_BERT_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT12000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_ICR": "",

    # T5
    "BOSS_Sentiment_SemEval_T5_Insert": "",
    "BOSS_Sentiment_SemEval_T5_Substitute": "",
    "BOSS_Sentiment_SemEval_T5_Translate": "",
    "BOSS_Sentiment_SemEval_T5_Paraphrase": "",
    "BOSS_Sentiment_SemEval_T5_ICR": "",

    # Falcon
    "BOSS_Sentiment_SemEval_Falcon_Insert": "",
    "BOSS_Sentiment_SemEval_Falcon_Substitute": "",
    "BOSS_Sentiment_SemEval_Falcon_Translate": "",
    "BOSS_Sentiment_SemEval_Falcon_Paraphrase": "",
    "BOSS_Sentiment_SemEval_Falcon_ICR": "",

    # Dynasent
    # BERT
    "BOSS_Sentiment_Dynasent_BERT_Insert": "",
    "BOSS_Sentiment_Dynasent_BERT_Substitute": "",
    "BOSS_Sentiment_Dynasent_BERT_Translate": "",
    "BOSS_Sentiment_Dynasent_BERT_Paraphrase": "",
    "BOSS_Sentiment_Dynasent_BERT_ICR": "",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Insert": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Translate": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_ICR": "",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT12000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_ICR": "",

    # T5
    "BOSS_Sentiment_Dynasent_T5_Insert": "",
    "BOSS_Sentiment_Dynasent_T5_Substitute": "",
    "BOSS_Sentiment_Dynasent_T5_Translate": "",
    "BOSS_Sentiment_Dynasent_T5_Paraphrase": "",
    "BOSS_Sentiment_Dynasent_T5_ICR": "",

    # Falcon
    "BOSS_Sentiment_Dynasent_Falcon_Insert": "",
    "BOSS_Sentiment_Dynasent_Falcon_Substitute": "",
    "BOSS_Sentiment_Dynasent_Falcon_Translate": "",
    "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": "",
    "BOSS_Sentiment_Dynasent_Falcon_ICR": "",

    # BOSS Toxicity
    # ID
    # BERT
    "BOSS_Toxicity_ID_BERT_Insert": "",
    "BOSS_Toxicity_ID_BERT_Substitute": "",
    "BOSS_Toxicity_ID_BERT_Translate": "",
    "BOSS_Toxicity_ID_BERT_Paraphrase": "",
    "BOSS_Toxicity_ID_BERT_ICR": "",

    # T5
    "BOSS_Toxicity_ID_T5_Insert": "",
    "BOSS_Toxicity_ID_T5_Substitute": "",
    "BOSS_Toxicity_ID_T5_Translate": "",
    "BOSS_Toxicity_ID_T5_Paraphrase": "",
    "BOSS_Toxicity_ID_T5_ICR": "",

    # Falcon
    "BOSS_Toxicity_ID_Falcon_Insert": "",
    "BOSS_Toxicity_ID_Falcon_Substitute": "",
    "BOSS_Toxicity_ID_Falcon_Translate": "",
    "BOSS_Toxicity_ID_Falcon_Paraphrase": "",
    "BOSS_Toxicity_ID_Falcon_ICR": "",

    # Toxigen
    # BERT
    "BOSS_Toxicity_Toxigen_BERT_Insert": "",
    "BOSS_Toxicity_Toxigen_BERT_Substitute": "",
    "BOSS_Toxicity_Toxigen_BERT_Translate": "",
    "BOSS_Toxicity_Toxigen_BERT_Paraphrase": "",
    "BOSS_Toxicity_Toxigen_BERT_ICR": "",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_ICR": "",

    # T5
    "BOSS_Toxicity_Toxigen_T5_Insert": "",
    "BOSS_Toxicity_Toxigen_T5_Substitute": "",
    "BOSS_Toxicity_Toxigen_T5_Translate": "",
    "BOSS_Toxicity_Toxigen_T5_Paraphrase": "",
    "BOSS_Toxicity_Toxigen_T5_ICR": "",

    # Falcon
    "BOSS_Toxicity_Toxigen_Falcon_Insert": "",
    "BOSS_Toxicity_Toxigen_Falcon_Substitute": "",
    "BOSS_Toxicity_Toxigen_Falcon_Translate": "",
    "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": "",
    "BOSS_Toxicity_Toxigen_Falcon_ICR": "",

    # ImplicitHate
    # BERT
    "BOSS_Toxicity_ImplicitHate_BERT_Insert": "",
    "BOSS_Toxicity_ImplicitHate_BERT_Substitute": "",
    "BOSS_Toxicity_ImplicitHate_BERT_Translate": "",
    "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": "",
    "BOSS_Toxicity_ImplicitHate_BERT_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_ICR": "",

    # T5
    "BOSS_Toxicity_ImplicitHate_T5_Insert": "",
    "BOSS_Toxicity_ImplicitHate_T5_Substitute": "",
    "BOSS_Toxicity_ImplicitHate_T5_Translate": "",
    "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": "",
    "BOSS_Toxicity_ImplicitHate_T5_ICR": "",

    # Falcon
    "BOSS_Toxicity_ImplicitHate_Falcon_Insert": "",
    "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": "",
    "BOSS_Toxicity_ImplicitHate_Falcon_Translate": "",
    "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": "",
    "BOSS_Toxicity_ImplicitHate_Falcon_ICR": "",

    # AdvCivil
    # BERT
    "BOSS_Toxicity_AdvCivil_BERT_Insert": "",
    "BOSS_Toxicity_AdvCivil_BERT_Substitute": "",
    "BOSS_Toxicity_AdvCivil_BERT_Translate": "",
    "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": "",
    "BOSS_Toxicity_AdvCivil_BERT_ICR": "",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_ICR": "",

    # T5
    "BOSS_Toxicity_AdvCivil_T5_Insert": "",
    "BOSS_Toxicity_AdvCivil_T5_Substitute": "",
    "BOSS_Toxicity_AdvCivil_T5_Translate": "",
    "BOSS_Toxicity_AdvCivil_T5_Paraphrase": "",
    "BOSS_Toxicity_AdvCivil_T5_ICR": "",

    # Falcon
    "BOSS_Toxicity_AdvCivil_Falcon_Insert": "",
    "BOSS_Toxicity_AdvCivil_Falcon_Substitute": "",
    "BOSS_Toxicity_AdvCivil_Falcon_Translate": "",
    "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": "",
    "BOSS_Toxicity_AdvCivil_Falcon_ICR": "",

    # AG News Tweets
    # ID
    # BERT
    "AgNewsTweets_ID_BERT_Insert": "",
    "AgNewsTweets_ID_BERT_Substitute": "",
    "AgNewsTweets_ID_BERT_Translate": "",
    "AgNewsTweets_ID_BERT_Paraphrase": "",
    "AgNewsTweets_ID_BERT_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_ICR": "",

    # T5
    "AgNewsTweets_ID_T5_Insert": "",
    "AgNewsTweets_ID_T5_Substitute": "",
    "AgNewsTweets_ID_T5_Translate": "",
    "AgNewsTweets_ID_T5_Paraphrase": "",
    "AgNewsTweets_ID_T5_ICR": "",

    # Falcon
    "AgNewsTweets_ID_Falcon_Insert": "",
    "AgNewsTweets_ID_Falcon_Substitute": "",
    "AgNewsTweets_ID_Falcon_Translate": "",
    "AgNewsTweets_ID_Falcon_Paraphrase": "",
    "AgNewsTweets_ID_Falcon_ICR": "",

    # Tweets
    # BERT
    "AgNewsTweets_Tweets_BERT_Insert": "",
    "AgNewsTweets_Tweets_BERT_Substitute": "",
    "AgNewsTweets_Tweets_BERT_Translate": "",
    "AgNewsTweets_Tweets_BERT_Paraphrase": "",
    "AgNewsTweets_Tweets_BERT_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_ICR": "",

    # T5
    "AgNewsTweets_Tweets_T5_Insert": "",
    "AgNewsTweets_Tweets_T5_Substitute": "",
    "AgNewsTweets_Tweets_T5_Translate": "",
    "AgNewsTweets_Tweets_T5_Paraphrase": "",
    "AgNewsTweets_Tweets_T5_ICR": "",

    # Falcon
    "AgNewsTweets_Tweets_Falcon_Insert": "",
    "AgNewsTweets_Tweets_Falcon_Substitute": "",
    "AgNewsTweets_Tweets_Falcon_Translate": "",
    "AgNewsTweets_Tweets_Falcon_Paraphrase": "",
    "AgNewsTweets_Tweets_Falcon_ICR": "",
}

print(json.dumps(table_paths, indent=4))

{
    "BOSS_Sentiment_ID_BERT_Insert": "",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "",
    "BOSS_Sentiment_ID_BERT_ICR": "",
    "BOSS_Sentiment_ID_T5_Insert": "",
    "BOSS_Sentiment_ID_T5_Substitute": "",
    "BOSS_Sentiment_ID_T5_Translate": "",
    "BOSS_Sentiment_ID_T5_Paraphrase": "",
    "BOSS_Sentiment_ID_T5_ICR": "",
    "BOSS_Sentiment_ID_Falcon_Insert": "",
    "BOSS_Sentiment_ID_Falcon_Substitute": "",
    "BOSS_Sentiment_ID_Falcon_Translate": "",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": "",
    "BOSS_Sentiment_ID_Falcon_ICR": "",
    "BOSS_Sentiment_SST5_BERT_Insert": "",
    "BOSS_Sentiment_SST5_BERT_Substitute": "",
    "BOSS_Sentiment_SST5_BERT_Translate": "",
    "BOSS_Sentiment_SST5_BERT_Paraphrase": "",
    "BOSS_Sentiment_SST5_BERT_ICR": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert": "",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute": "",
    "BOSS_Sent

In [ ]:
run = wandb.init()

In [ ]:
def get_local_logs_path(directory):
    try:
        for contents in os.listdir(directory):
            if contents.endswith("style_logs.table.json"):
                return os.path.join(directory, contents)

            if os.path.isdir(os.path.join(directory, contents)):
                return get_local_logs_path(os.path.join(directory, contents))
    except:
        # print("No style logs found in {}".format(directory))
        return None

print(get_local_logs_path("data/BOSS_Toxicity_ID_Falcon_Paraphrase"))

In [ ]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for table_name, table_path in tqdm(list(table_paths.items())[:]):
    # if table_name == "BOSS_Toxicity_ImplicitHate_BERT_ICR":
    #     print()
    # else:
    #     continue

    try:
        if table_path == "":
            missing_tables.append(table_name)
            continue

        if get_local_logs_path(f"data/{table_name}") is None:
            download_path = f"{project_name}/{table_path}"
            print(f"Downloading {table_name} --- {download_path}")
            artifact = run.use_artifact(download_path, type="run_table")
            artifact_dir = artifact.download(root=f"data/{table_name}")

        local_paths[table_name] = get_local_logs_path(f"data/{table_name}")
    except Exception as e:
        print(e)
        print(table_name)
        missing_tables.append(table_name)

print(json.dumps(missing_tables, indent=4))

In [ ]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "generations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [ ]:
def parse_augmentations(augmentations):
    if augmentations is None or len(augmentations) == 0:
        return []

    return [aug.replace("<aug>", "").replace("</aug>", "") for aug in augmentations[:4]]

formatted_augs = parse_augmentations([ "<aug>too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.</aug>", "<aug>one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.</aug>", "<aug>too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.</aug>", "<aug>too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>" ])

print(formatted_augs)
print(len(formatted_augs))

In [ ]:
local_paths

In [ ]:
combined_datset = DatasetDict()
for split in tqdm(local_paths):
    # print(f"Processing {split}")
    path = local_paths[split]
    json_logs = json.load(open(path))
    frame = pd.DataFrame(data=json_logs["data"], columns=json_logs["columns"])
    frame = frame.rename(columns=column_renaming)
    frame = frame.drop(columns=columns_to_drop)
    frame["augmentations"] = frame["augmentations"].apply(parse_augmentations)

    if "BERT" not in split:
        for col in columns_order:
            if col not in frame.columns:
                if col == "original_prediction_entropy" or col == "tta_prediction_entropy" or col == "prediction_entropy_decrease":
                    frame[col] = float("nan")
                elif col == "prediction_entropy_decreased":
                    frame[col] = False
                elif col == "tta_all_class_probs":
                    frame[col] = frame.apply(lambda x: [[float(0.0)]], axis=1)
                elif "probs" in col:
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)
                elif col == "tta_all_class_entropy":
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)

    if "generations" not in frame.columns:
        frame["generations"] = [[""]] * len(frame)

    frame = frame[columns_order]
    combined_datset[split] = Dataset.from_pandas(frame)

combined_datset

In [ ]:
combined_datset.save_to_disk("data/combined_dataset")

In [ ]:
list(combined_datset.keys())

In [ ]:
assert len(combined_datset["BOSS_Toxicity_ID_BERT_Insert"].features) == len(combined_datset["BOSS_Toxicity_ID_T5_Insert"].features)
for feature, value_data in combined_datset["BOSS_Toxicity_ID_BERT_Insert"].features.items():
    bert_dtype = value_data.dtype
    t5_dtype = combined_datset["BOSS_Toxicity_ID_T5_Insert"].features[feature].dtype
    error_message = f"Feature {feature} has different dtypes for BERT and T5: {bert_dtype} and {t5_dtype}"
    assert bert_dtype == t5_dtype, error_message

In [ ]:
combined_datset["BOSS_Sentiment_ID_BERT_ICR"].features

In [ ]:
combined_datset["BOSS_Sentiment_ID_T5_Insert"].features

In [ ]:
# combined_datset.push_to_hub("Kyle1668/LLM-TTA-Augmentation-Logs")

In [ ]:
# combined_datset.push_to_hub("Kyle1668/LLM-TTA-Augmentation-Logs")


ValueError: All datasets in `DatasetDict` should have the same features but features for 'BOSS_Sentiment_ID_BERT_ICR' and 'BOSS_Sentiment_ID_T5_Insert' don't match: 

{'outcome': Value(dtype='string', id=None), 
'original_text': Value(dtype='string', id=None), 
'augmentations': Sequence(feature=Value(dtype='string', id=None),length=-1, id=None), 'generations': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None), 'original_predicted_class': Value(dtype='int64', id=None), 'tta_predicted_class': Value(dtype='int64', id=None), 'label': Value(dtype='int64', id=None), 'tta_inference_latency': Value(dtype='float64', id=None), 'original_prediction_entropy': Value(dtype='float64', id=None), 'tta_prediction_entropy': Value(dtype='float64', id=None), 'prediction_entropy_decreased': Value(dtype='bool', id=None), 'prediction_entropy_decrease': Value(dtype='float64', id=None), 'tta_mean_class_probs': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'tta_all_class_probs': Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None), 'tta_all_class_entropy': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)} != 

{'outcome': Value(dtype='string', id=None), 'original_text': Value(dtype='string', id=None), 'augmentations': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'generations': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'original_predicted_class': Value(dtype='int64', id=None), 'tta_predicted_class': Value(dtype='int64', id=None), 'label': Value(dtype='int64', id=None), 'tta_inference_latency': Value(dtype='float64', id=None), 'original_prediction_entropy': Value(dtype='float64', id=None), 'tta_prediction_entropy': Value(dtype='float64', id=None), 'prediction_entropy_decreased': Value(dtype='bool', id=None), 'prediction_entropy_decrease': Value(dtype='float64', id=None), 'tta_mean_class_probs': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), 'tta_all_class_probs': Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None), 'tta_all_class_entropy': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}"